In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [2]:
import yfinance as yf

In [3]:
def get_ticker_data(ticker, startdate, enddate):
    ydata = yf.Ticker(ticker)
    rdata = ydata.history(period="2y") # 오늘부터 2년치
    rdata.reset_index('Date', inplace=True)
    rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()
    rdata = rdata[(rdata['Date'] <= enddate) & (rdata['Date'] >= startdate)] 
    rdata['temp'] = rdata['Close'].shift(1)
    val_temp = (rdata['Close'] - rdata['temp'])/rdata['temp']*100
    rdata['close_cr'] = val_temp.map("{:.2f}".format)
    rdata = rdata[['Date','Open', 'High', 'Low', 'Close', 'Volume', 'close_cr']] # 필요한 column만 남김
    rdata.columns = ['date', 'open', 'high', 'low', 'close', 'vol', 'close_cr'] # column이름 통일
    rdata.reset_index(drop=True, inplace=True) # index번호를 0부터 재정리
    return rdata

In [4]:
start_date = datetime.datetime(2021,12,20).date()
# ed = end_date = datetime.datetime(2021,12,31).date()
end_date = datetime.datetime.today().date()

In [5]:
kakao = get_ticker_data('035720.KS', start_date, end_date)

In [9]:
kakao.tail()

,date,open,high,low,close,vol,close_cr
315,2023-04-03,61400.0,61600.0,59900.0,60200.0,1683150,-1.47
316,2023-04-04,60200.0,60800.0,59800.0,60700.0,1380518,0.83
317,2023-04-05,60500.0,61200.0,60200.0,60500.0,1227329,-0.33
318,2023-04-06,60100.0,60100.0,58500.0,58600.0,2659408,-3.14
319,2023-04-07,58600.0,59200.0,58100.0,58600.0,1644048,0.00


In [7]:
kk = pd.read_pickle('kakao_historical.pkl')

In [8]:
kk.tail()

,date,open,high,low,close,close_cr,vol
456,2023-04-02,,,,,,
457,2023-04-03,61400,61600,59900,60200,-1.47,101978
458,2023-04-04,60200,60800,59800,60700,0.83,83243
459,2023-04-05,60500,61200,60200,60500,-0.33,74433
460,2023-04-06,60100,60100,58500,58600,-3.14,156394


### yahoo는 거래량. 내가 수집한 데이터는 거래금액임. 맞추어야 함. 